In [3]:
# Getting the input data
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from scipy import stats 
import shutil


In [4]:
os.chdir('/Users/nicolethomas/Google Drive/Grad School/IACMI Project/Python')

import os

os.listdir('.')

['.DS_Store',
 '.ipynb_checkpoints',
 'Compression_Analysis_of_Composites.ipynb',
 'Eng_Strain_Stress.tiff',
 'ey_strain.csv',
 'ey_strain_plotted.xlsx',
 'ey_strain_plotted_with_Stress.xlsx',
 'force_strain_plotted.xlsx',
 'Grocery list test.rtf',
 'Grocery List.ipynb',
 'HX1_no1_02_analysis1.csv',
 'specimen.csv',
 'Tensile with extra heading.xlsx',
 'Tensile.csv',
 'Tensile.dat',
 'Tensile_Analysis_of_Composites.ipynb',
 'Tensile_analysis_orange.ows',
 'Tensile_modified.xlsx',
 'Tensile_no_headings.csv',
 'Tensile_Plot_Aruvi.ipynb',
 '~$ey_strain_plotted_with_Stress.xlsx',
 '~$Tensile_modified.xlsx']

In [5]:
os.chdir('/Users/nicolethomas/Google Drive/Grad School/IACMI Project/Python')

import os

os.listdir('.')

# Reading all the data in csv format to the data frame

output = pd.read_csv("specimen.csv",skiprows = 8,names = ['Time','Force','Disp']).dropna(axis = 0)
output = output.apply(pd.to_numeric, errors ='coerce')
eystrain = pd.read_csv("ey_strain.csv",skiprows = 3,names =['Stage','ey_strain']).dropna(axis =0)
eystrain = eystrain.apply(pd.to_numeric, errors='coerce')
#mises = pd.read_csv("mises.csv",skiprows = 3, names =['Stage','mises_strain']).dropna(axis = 0)
#mises = mises.apply(pd.to_numeric, errors='coerce')
#Dimensions = pd.read_table("Dimensions.txt")

print('Input_length:')
print(len(eystrain))
print(output.head(5))

Input_length:
836
                                                 Time  Force  Disp
Data Acquisition NaN NaN NaN NaN NaN Time:  409.36328    NaN   NaN


In [6]:
#Calculating Stress from the output data 
Area = int(Dimensions["Width"]*Dimensions["Thickness"])

# Changing strain from percentage to mm/mm format
eystrain["ey_strain"] = eystrain["ey_strain"]/100
mises["mises_strain"] = mises["mises_strain"]/100

# Calculating Stress from force 
output["stress"] = output["Force"]/Area 

# Assembling all the data to a data frame
data = pd.concat(
    [output['Time'], output['stress'], eystrain['Stage'], eystrain['ey_strain'], mises['mises_strain']], 
    axis = 1, keys= ['time','stress','stage','strain','mises'])

print(data.head(10))


NameError: name 'Dimensions' is not defined

In [ ]:
#Filtering stress

#Filtering out the Stress data corresponding to each Strain Stage

Stress = []
time = np.array(data['time'])
stage = np.array(data['stage'].dropna())
time_sorted = []

# Finding closest time value of stress corresponding to each strain stage data
for i in range(0, len(stage)):
    time_sorted.append(min(range(len(time)), key=lambda j: abs(time[j]-stage[i])))

for i in range (0, len(time_sorted)):
        Stress.append(data['stress'][time_sorted[i]]) 

Strain = np.array(data["strain"].dropna())
Mises = np.array(data["mises"].dropna())

print(len(Strain))
print(len(Stress))



# Finding Split point 

In [ ]:
def Split_Stress(stress, strain):
    
    # Split_point is strain at whaich standard deviation(ymodulus)/mean(ymodulus) > 10 %
    
    ymodulus = []
    
    for i in range(0,len(strain)-10):
        ymodulus.append((stress[i+10] - stress[i])/(strain[i+10] - strain[i]))
        m = np.mean(ymodulus)
        s = np.std(ymodulus)
        p = (s/m)*100
        if np.floor(p) > 10:
            global Split_Point
            Split_Point = strain[i]  
            break

    #Calculating Young's Modulus from raw data ey_strain

    #Filtering out the linear potion of the ey_strain and stress data to find the slope

    linear_strain = []
    linear_stress = []
    strain_deviation = []

    for i in range(0, len(strain)):
        if strain[i] < Split_Point:
            linear_strain.append(strain[i])
            linear_stress.append(stress[i])
    
    linear_strain = np.array(linear_strain)
    linear_stress = np.array(linear_stress)

    linear_strain = linear_strain[:,np.newaxis]
           
    E_trial,r,_,_ = np.linalg.lstsq(linear_strain,linear_stress)    # Finding Young's Modulus by forcing to zero intercept
    
    # Calculating mean strain deviation for the linear_stress, linear_strain data
    
    for i in range(0,len(linear_strain)):
        strain_deviation.append((linear_strain[i] - linear_stress[i]/E_trial))
        
    mean_strain_deviation = np.mean(strain_deviation)
    
    # Filtering out the stress, strain data with strain deviation within bounds 
    # of mean strain deviation +/- Extensometer least count
            
    filtered_stress =[]
    
    for i in range(0,len(strain)):
            if mean_strain_deviation-0.00005 < (strain[i]- (stress[i]/E_trial)) < mean_strain_deviation + 0.00005 :
                filtered_stress.append(stress[i])
    break_stress = np.max(filtered_stress)
    return(break_stress)
print('done')

# Finding Mechanical Properties 

In [ ]:
# Function to calculate Mechanical Properties

def Mechanical_Properties(stress, strain, split_stress):

    #Calculating Young's Modulus from raw data ey_strain

    #Filtering out the linear potion of the ey_strain and stress data to find the slope

    linear_strain = []
    linear_stress = []

    for i in range(0, len(stress)):
        if stress[i] < split_stress:
            linear_strain.append(strain[i])
            linear_stress.append(stress[i])

    # Filtering out the linear potion of the ey_strain and stress data to fit a polynomial

    nonlinear_strain =[]
    nonlinear_stress =[]

    for i in range(0,len(stress)):
        if stress[i] > split_stress:
            nonlinear_strain.append(strain[i])
            nonlinear_stress.append(stress[i])
            
    linear_strain = np.array(linear_strain)
    linear_stress = np.array(linear_stress)

    linear_strain = linear_strain[:,np.newaxis]

    E,r,_,_ = np.linalg.lstsq(linear_strain,linear_stress)
    

    #Calculating R^2 value
    K = len(linear_stress)
    r_squared =  1 - r /(K * linear_stress.var())

    #Calculating coefficient of variation
    V = 100*np.sqrt(((1/r_squared)-1)/(K-2))
    
    
    #Fitting a polynomial curve to the nonlinear portion of stress-strain curve
    #p = np.polyfit(nonlinear_strain,nonlinear_stress,20)
    #p1d = np.poly1d(p)
    
    
    #Finding Ultimate Stress and Strain
    ucs = np.amax(stress)
    ucstrain = np.max(strain)
    #oldcode  ucs = np.amax(stress)    ucstrain = np.amax(strain)


    # To find yield stress from initially filtered data of Engineering strain vs Engineering stress
    # This works by finding intersection point of the straight line and polynomial

    x = np.linspace(0.002,0.05,100000)

    for i in range(0,len(x)):
        if (E*(x[i] -0.002)) > (p1d(x[i])):
            yieldstrain = x[i]
            yieldstress = p1d(x[i])
            break 

    #Caculating ductility from filtered data ey_strain
        
    #ductility = (ucstrain - yieldstrain)*100
    return(yieldstress,ucs,E,ductility,r_squared,K,V) #Old code return(yieldstress,ucs,E,ductility,r_squared,K,V)

print("done")


In [ ]:
#Mechanical properties in different processing conditions

uniaxial_split = Split_Stress(Stress,Strain)   # Passing uniaxial strain, uniaxial stress
print(uniaxial_split)
mises_split = Split_Stress(Stress, Mises)      # Passing mises strain, uniaxial stress
print(mises_split)

[y,ucs,E,d,R2,K,V] = Mechanical_Properties(Stress,Strain,uniaxial_split)


[y1,ucs1,E1,d1,R2_1,K1,V1] = Mechanical_Properties(Stress,Mises,mises_split)

dir = 'Results'
if not os.path.exists(dir):
    os.makedirs(dir)
else:
    shutil.rmtree(dir)           #removes all the subdirectories!
    os.makedirs(dir)

os.chdir('Results')



Data = pd.DataFrame({'Strain':Strain,'Mises':Mises,'Stress':Stress},columns = ['Strain','Mises','Stress'])

Results = pd.DataFrame({'Result Type':['Uniaxial','Mises'],'Yield Strength(Mpa)':[y,y1],'Ultimate Strength(Mpa)':[ucs,ucs1],
                        'Youngs Modulus(Gpa)':[E/1000,E1/1000],'Ductility':[d,d1],'R^2':[R2,R2_1],'Data Points':[K,K1],'CoV(%)':[V,V1]},
                        columns = ['Result Type','Yield Strength(Mpa)','Ultimate Strength(Mpa)','Youngs Modulus(Gpa)',
                                   'Ductility','R^2','Data Points','CoV(%)'])

Results.to_csv('Results.csv')
Data.to_csv('Data.csv')

print(Results)


In [ ]:
# Plotting 

# To draw straight line in graph

b = np.linspace(0,0.01,1000)
g = np.linspace(0.002,0.015,2000)

a = E*b
c = E*(g - 0.002)

a1 = E1*b
c1 = E1*(g - 0.002)

#plt.clf()

plt.figure(figsize = (20,15))
plt.rc('xtick', labelsize=20) 
plt.rc('ytick', labelsize=20)
plt.plot(Strain, Stress, 'k.')
plt.plot(b,a,'-')
plt.plot(g,c,'-')
plt.xlabel('Engineering Strain (mm/mm)', fontsize = 20)
plt.ylabel('Engineering Stress (Mpa)', fontsize = 20)
plt.xlim([0,0.02])
plt.ylim([0,1500])
plt.savefig('Eng_Strain_Stress.tiff')

plt.figure(figsize = (20,15))
plt.rc('xtick', labelsize=20) 
plt.rc('ytick', labelsize=20)
plt.plot(Mises, Stress, 'k.')
plt.plot(b,a1,'-')
plt.plot(g,c1,'-')
plt.xlabel('Mises Strain (mm/mm)', fontsize = 20)
plt.ylabel('Engineering Stress (Mpa)', fontsize = 20)
plt.xlim([0,0.02])
plt.ylim([0,1500])
plt.savefig('Mises_Strain_Stress_Galvanized.tiff')

plt.show()


In [ ]:
#os.chdir('..')
#os.getcwd()
#for d in os.listdir(os.getcwd()):
#    os.chdir(d)
#    os.mkdir('RREESSUULLTTSS')
#    os.chdir('..')

In [ ]:
#os.chdir('E:\Python\Tensile_Plot\Data')
#os.getcwd()

In [ ]:
# ASSUME IN SAME FOLDERS AS SAMPLE FOLDERS (i.e. A09)
#samples = glob.glob('A*')

#for sample in samples:
    # change such that you can list the csv folders for a single sample
    
    # List csv files
    #strain_files = glob.glob(sample+'/*csv')
    
    
    # Call conversion function
    #df = read_csv_to_df(strain_files[0], strain_files[1], strain_files[2])
    
    # Call analysis
    #results = analysis_function(df)
    
    
    # Write results
    

